In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
import glob

def teager_kaiser_energy_operator(signal):
    """
    Calculates the Teager-Kaiser Energy Operator (TKEO) for a 1D signal.
    """
    tkeo_signal = np.zeros_like(signal, dtype=float)
    for i in range(1, len(signal) - 1):
        tkeo_signal[i] = signal[i]**2 - (signal[i+1] * signal[i-1])
    return tkeo_signal

def apply_tkeo_to_feature_matrix(feature_matrix):
    """
    Applies TKEO to each row (feature dimension) of a 2D feature matrix.
    """
    result = []
    for row in feature_matrix:
        tkeo_row = teager_kaiser_energy_operator(row)
        result.append(tkeo_row)
    return np.array(result)

# === Load MATLAB files ===
feature_files = sorted(glob.glob('features_sample*.mat'))

for f in feature_files:
    print(f"\n--- {f} ---")
    data = loadmat(f)

    # Load each feature matrix
    mfcc     = data['mfcc']
    centroid = data['centroid']
    rolloff  = data['rolloff']
    mel      = data['mel']
    chroma   = data['chroma']

    # Apply TKEO to each feature type
    features = {
        "MFCC": apply_tkeo_to_feature_matrix(mfcc),
        "Centroid": apply_tkeo_to_feature_matrix(centroid),
        "Rolloff": apply_tkeo_to_feature_matrix(rolloff),
        "Mel": apply_tkeo_to_feature_matrix(mel),
        "Chroma": apply_tkeo_to_feature_matrix(chroma)
    }

    # Compute and display skewness and kurtosis
    for name, feat_tkeo in features.items():
        sk = skew(feat_tkeo, axis=1)
        ku = kurtosis(feat_tkeo, axis=1)
        print(f"{name} - Skewness (TKEO):\n{np.round(sk, 4)}")
        print(f"{name} - Kurtosis (TKEO):\n{np.round(ku, 4)}\n")
